In [19]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import xml.etree.ElementTree as ET
import napari
import tifffile as tiff
from PIL import Image
from io import StringIO
from aicsimageio import AICSImage

20-Jan-25 18:48:29 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


In [33]:
def print_metadata_structure(element, indent=0):
    print("  " * indent + f"Tag: {element.tag}")
    if element.attrib:
        print("  " * (indent + 1) + "Attributes:")
        for key, value in element.attrib.items():
            print("  " * (indent + 2) + f"{key}: {value}")
    for child in element:
        print_metadata_structure(child, indent + 1)

In [30]:
ome_path = '/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/breast_fibro_IMC/images/multistack_tiffs/20201013_LC_BC-Fibro_TBB075_s0_p8_r1_a1_ac.ome.tiff'
img = tiff.imread(ome_path)
seg = tiff.imread('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/breast_fibro_IMC/masks/Cell Masks/20201013_LC_BC-Fibro_TBB075_s0_p8_r1_a1_ac_ilastik_s2_Probabilities_cellscell_mask.tiff')

In [4]:
img.shape

(135, 1000, 1000)

In [59]:
with tiff.TiffFile(ome_path) as tif:
    ome_metadata = tif.ome_metadata
    
    if ome_metadata:
        root = ET.fromstring(ome_metadata)
        print("Complete OME Metadata Structure:")
        print_metadata_structure(root)
    else:
        print("No OME metadata found in the file.")

Complete OME Metadata Structure:
Tag: {http://www.openmicroscopy.org/Schemas/ome/2013-06s}OME
  Attributes:
    {http://www.w3.org/2001/XMLSchema-instance}schemaLocation: http://www.openmicroscopy.org/Schemas/OME/2013-06 http://www.openmicroscopy.org/Schemas/OME/2012-03/ome.xsd
  Tag: {http://www.openmicroscopy.org/Schemas/ome/2013-06s}Image
    Attributes:
      ID: Image:0
      Name: 20201013_LC_BC-Fibro_TBB075_s0_p8_r1_a1_ac.ome.tiff
    Tag: {http://www.openmicroscopy.org/Schemas/ome/2013-06s}AcquisitionDate
    Tag: {http://www.openmicroscopy.org/Schemas/ome/2013-06s}Pixels
      Attributes:
        DimensionOrder: XYCZT
        ID: Pixels:0
        SizeC: 135
        SizeT: 1
        SizeX: 1000
        SizeY: 1000
        SizeZ: 1
        Type: float
      Tag: {http://www.openmicroscopy.org/Schemas/ome/2013-06s}Channel
        Attributes:
          Fluor: As75
          ID: Channel:0:0
          Name: 75As
          SamplesPerPixel: 1
        Tag: {http://www.openmicroscopy.or

In [54]:
with tiff.TiffFile('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/breast_fibro_IMC/images/multistack_tiffs/20201013_LC_BC-Fibro_TBB075_s0_p8_r1_a1_ac.ome.tiff') as tif:
    md = tif.ome_metadata
    #channel_names = [channel.get('Name') for channel in tif.md.findall('.//Channel')]
    root = ET.fromstring(md)
    ns = {'ome': 'http://www.openmicroscopy.org/Schemas/ome/2013-06s'}
    channels = root.findall('ome:Image/ome:Pixels/ome:Channel', ns)
    channel_names = [channel.get('Name') for channel in channels]

In [61]:
channel_names

['75As',
 '76Se',
 '77Se',
 '78Se',
 '79Br',
 '80ArAr',
 '81Br',
 '82Kr',
 '83Kr',
 '84Sr',
 '85Rb',
 '86Sr',
 '87Sr',
 '88Sr',
 'Myelope_2762871Y89',
 '90Zr',
 '91Zr',
 '92Zr',
 '93Nb',
 '94Zr',
 '95Mo',
 '96Mo',
 '97Mo',
 '98Mo',
 '99Ru',
 '100Ru',
 '101Ru',
 '102Ru',
 '103Rh',
 '104Pd',
 '105Pd',
 '106Pd',
 '107Ag',
 '108Pd',
 '109Ag',
 '110Cd',
 '111Cd',
 '112Cd',
 '113Cd',
 'FSP1 S1_2402384In113',
 '114Cd',
 'SMA_1743277In115',
 '116Sn',
 '117Sn',
 '118Sn',
 '119Sn',
 '120Sn',
 '121Sb',
 '122Sn',
 '123Te',
 '124Te',
 '125Te',
 '126Te',
 '127I',
 '128Xe',
 '129Xe',
 '130Xe',
 '131Xe',
 '132Xe',
 '133Cs',
 '134Xe',
 '135Ba',
 '136Ba',
 '137Ba',
 '138Ba',
 '139La',
 '140Ce',
 'Histone_1262797Pr141',
 'CD11b_22793026Nd142',
 'HLA-DR_18492792Nd143',
 'CD146_223259Nd144',
 'Cadheri_20882893Nd145',
 'fap_3232799Nd146',
 'CD11b_92915Sm147',
 'VCAM1_19862801Nd148',
 'CD20_362730Sm149',
 'CD68_772864Nd150',
 'Indolea_22813014Eu151',
 'CD3_18413018Sm152',
 'Podopla_14632619Eu153',
 'CD11c_24

In [63]:
with open('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/breast_fibro_IMC/markers.txt', 'w', encoding='utf-8') as f:
    f.writelines([str(i) + '\n' for i in channel_names])

In [60]:
seg

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [ ]:
array = np.array(seg, dtype=np.uint32)
array.max()

In [ ]:
array

In [ ]:
array[1000,2000]

In [ ]:
len(np.unique(array))

In [ ]:
img = imread('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/lymphoma_CODEX/segmentation/191_1/reg001_X01_Y01_Z01_visual_us4.0_grow0.0x1.0x2.5b2.0_labeled.png')
img

In [ ]:
img.max()

In [ ]:
len(np.unique(img))

In [ ]:
len(img)